# 如何使用SmartLLMChain

SmartLLMChain是一种自我批判链，可以帮助您回答特别复杂的问题。与进行单次LLM传递不同，它执行以下3个步骤：
1. 构思：将用户提示通过LLM传递n次，以获取n个输出提案（称为“想法”），其中n是您可以设置的参数
2. 批判：LLM批判所有想法，找出可能的缺陷并选择最佳想法
3. 解决：LLM试图改进最佳想法（在批判步骤中选择）并输出它。然后，这就是最终的输出。

SmartLLMChain基于在https://youtu.be/wVzuvf9D9BU中提出的SmartGPT工作流程。

请注意，SmartLLMChain
- 使用更多的LLM传递（即n+2而不仅仅是1次）
- 仅在底层LLM具有反思能力时才起作用，而较小的模型通常没有这种能力
- 仅适用于返回恰好1个输出而不是多个的底层模型

本笔记本演示了如何使用SmartLLMChain。

##### 所有步骤使用相同的LLM

In [1]:
# 导入os模块
import os

# 设置环境变量OPENAI_API_KEY为指定的值
os.environ["OPENAI_API_KEY"] = "..."

In [2]:
# 导入所需模块
from langchain.prompts import PromptTemplate
from langchain_experimental.smart_llm import SmartLLMChain
from langchain_openai import ChatOpenAI

作为示例问题，我们将使用“我有一个12升的水壶和一个6升的水壶。我想量出6升水。我该怎么做？”这是原始SmartGPT视频中的一个例子（https://youtu.be/wVzuvf9D9BU?t=384）。虽然这似乎是一个非常简单的问题，但LLMs在涉及数字和物理推理的这类问题上很难做出回答。

正如我们将看到的，所有三个初始想法都是完全错误的 - 即使我们使用的是GPT4！只有当使用自我反思时，我们才能得到正确的答案。

In [ ]:
# 定义一个字符串变量
hard_question = "I have a 12 liter jug and a 6 liter jug. I want to measure 6 liters. How do I do it?"

所以，我们首先创建一个LLM和提示模板。

In [3]:


# 创建一个PromptTemplate对象，从hard_question模板中获取问题
prompt = PromptTemplate.from_template(hard_question)

# 创建一个ChatOpenAI对象，设置温度为0，模型名称为"gpt-4"
llm = ChatOpenAI(temperature=0, model_name="gpt-4")

现在我们可以创建一个SmartLLMChain

In [4]:
# 创建一个 SmartLLMChain 对象
chain = SmartLLMChain(llm=llm, prompt=prompt, n_ideas=3, verbose=True)

现在我们可以将SmartLLM作为LLM的即插即用替代品。例如：

In [5]:


# 调用chain模块中的invoke函数，并传入一个空字典作为参数
chain.invoke({})



> Entering new SmartLLMChain chain...
Prompt after formatting:
I have a 12 liter jug and a 6 liter jug. I want to measure 6 liters. How do I do it?
Idea 1:
1. Fill the 6-liter jug completely.
2. Pour the water from the 6-liter jug into the 12-liter jug.
3. Fill the 6-liter jug again.
4. Carefully pour the water from the 6-liter jug into the 12-liter jug until the 12-liter jug is full.
5. The amount of water left in the 6-liter jug will be exactly 6 liters.
Idea 2:
1. Fill the 6-liter jug completely.
2. Pour the water from the 6-liter jug into the 12-liter jug.
3. Fill the 6-liter jug again.
4. Carefully pour the water from the 6-liter jug into the 12-liter jug until the 12-liter jug is full.
5. Since the 12-liter jug is now full, there will be 2 liters of water left in the 6-liter jug.
6. Empty the 12-liter jug.
7. Pour the 2 liters of water from the 6-liter jug into the 12-liter jug.
8. Fill the 6-liter jug completely again.
9. Pour the water from the 6-liter jug into the 12-liter j

'1. Fill the 12-liter jug completely.\n2. Pour the water from the 12-liter jug into the 6-liter jug until the 6-liter jug is full.\n3. The amount of water left in the 12-liter jug will be exactly 6 liters.'

##### 不同的LLM用于不同的步骤

您还可以通过传递 `ideation_llm`、`critique_llm` 和 `resolve_llm` 来为不同的步骤使用不同的LLM。您可能希望这样做，以便在构思阶段使用更具创造性（即高温度）的模型，而在批判和解决阶段使用更严格（即低温度）的模型。

In [8]:
# 创建一个SmartLLMChain对象，包括ideation_llm和llm两个ChatOpenAI实例
# ideation_llm用于生成创意，temperature为0.9，模型名称为"gpt-4"
# llm用于批判和解决问题，temperature为0，模型名称为"gpt-4"
# prompt为给定的提示
# n_ideas为生成创意的数量，设置为3
# verbose为True，表示输出详细信息
chain = SmartLLMChain(
    ideation_llm=ChatOpenAI(temperature=0.9, model_name="gpt-4"),
    llm=ChatOpenAI(
        temperature=0, model_name="gpt-4"
    ),  # 如果没有特定的llms，将用于批判和解决问题
    prompt=prompt,
    n_ideas=3,
    verbose=True,
)